# Sinha / Saha exploration

> From 4:28 mark of [New Recipe for Pi - Numberphile](https://youtu.be/nXexsSWrc1Q?t=268s)

He also discusses values for lambda (λ) at 605s (10:05).

`SinhaSaha:=(N, lambda) = 4+sum(1/k!*(1/(k+lambda)-4/(2*k+1))*pochhammer((2*k+1)^2/4/(k+lambda)-k, k-1), k=1..N);`


> Snippet from the appendix in the paper published by Sinha / Saha

![Appendix snippet](./snippet.svg)


In [ ]:
import logging
import sys

import cached_factorial as cf
import numpy as np
import scipy as sp
import sympy as sym
from latex_utils import set_display_level, sym_display_latex

N = 400
cf.set_cache_size(N)
cf.set_progress_frequency(np.floor(N / 10))

cf.cached_factorial.cache_reset()

set_display_level(logging.INFO)

sym.init_printing()


MILLION = 1_000_000
sys.set_int_max_str_digits(MILLION)


def mprint(msg: str, basic: sym.Basic | str | None = None, prec: int | None = None) -> None:
    if prec is not None:
        print(msg, end=' ')
        basic_str = str(basic) if type(basic) != str else basic
        prec = len(basic_str) if prec == -1 else prec
        print(f'>>{basic_str[:prec]}<<', end='')
        print(f'{basic_str[prec:]}')
    else:
        print(msg)
        if basic is not None:
            sym_display_latex('', basic)

> Load control file for 1 million pi digits (pi_1000000.txt) to validate how we are using sympy.pi

In [6]:
# read control file for 1_000_000 digits of pi

from pi_1000000 import read_pi_control

control_pi = read_pi_control('./pi_1000000.txt', MILLION)


def first_index_not_matched(s1: str, s2: str) -> int:
    for i, (c, p) in enumerate(zip(s1, s2)):
        if c != p:
            return i
    return -1

In [ ]:
PREC = MILLION

DEBUG_PI = True

if DEBUG_PI:
    prec_pi = sym.pi.evalf(PREC)
    prec_control_pi = control_pi.evalf(PREC)

    accuracy = (prec_control_pi - prec_pi).evalf(PREC/2)
    digits: int = int(sym.Abs(sym.floor(sym.log(accuracy) / sym.log(10))))

    digits_match_until = first_index_not_matched(str(prec_control_pi), str(prec_pi))
    all_digits_matched = digits_match_until == -1

    # subtract 1 from len for decimal point ('.')
    mprint(f'len control_pi: {len(str(prec_control_pi))-1:_}')
    mprint(f'len pi: {len(str(prec_pi))-1:_}')

    mprint(f'do all digits match? {all_digits_matched}')
    mprint(f'{digits_match_until=:_}')

    # mprint('control_pi:\t', str(prec_control_pi), digits)
    # mprint('pi:\t\t', str(prec_pi), digits)

    # mprint('control_pi - pi:', accuracy)

    mprint(f'\n{digits} digits accuracy')

## sympy representation of SinhaSaha

In [ ]:
n = sym.symbols('n', integer=True, positive=True)
λ, π = sym.symbols('λ π', real=True)
SinhaSaha = sym.symbols('SinhaSaha', cls=sym.Function)

sum_left_no_factorial = 1 / (n+λ) - 4 / (2*n + 1)
sum_right = (2*n + 1)**2 / (4 * (n + λ)).factor() - n

sum_inner = sum_left_no_factorial * sym.RisingFactorial(sum_right, n-1)
sum_inner

In [ ]:
nterms = sym.symbols('nterms', integer=True, positive=True)
# nterms = sym.oo
# nterms = sym.Integer(400)
# nterms = sym.Integer(PREC)

# def reciprocal_factorial(n: int):
#     return 1 / cf.cached_factorial(n)

SinhaSaha = 4 + sym.Sum((1 / sym.factorial(n)) * sum_inner, (n, 1, nterms))
SinhaSaha

In [ ]:
N = 400
λ_val = 575
prec = 130

SinhaSaha_subs = SinhaSaha.subs({λ: λ_val, nterms: N})
SinhaSaha_subs

In [ ]:
SinhaSaha_complete = sym.Eq(π, SinhaSaha_subs)
SinhaSaha_complete

In [ ]:
SinhaSaha_prec = SinhaSaha_subs.evalf(prec)
SinhaSaha_prec

In [ ]:
mprint(f'{N=}')
mprint(f'{λ_val=}')
mprint(f'{prec=}')
print()

pi = sym.pi.evalf(prec)
accuracy = (SinhaSaha_prec - pi).evalf(prec/2)
digits: int = int(sym.Abs(sym.floor(sym.log(accuracy) / sym.log(10))))

mprint('control_pi:\t', str(control_pi.evalf(prec)), digits)
mprint('pi:\t\t', str(pi), digits)

mprint('SinhaSaha:\t', str(SinhaSaha_prec), digits)

mprint('\nSinhaSaha - pi:', accuracy)

mprint(f'\n{digits} digits accuracy')